# ezSegmenter

#### This segmentation tool enables creation of masks for objects not easily picked up by primary cell segmentation methods on multiplexed imaging data.
#### In addition this tool can be used to create composites of channels as well as merge object masks with cell masks.

## 0: Set root directory and download example dataset

Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder.

* `base_dir`: the path to all of your imaging data. This directory will contain all of the data generated by this notebook.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [6]:
import pathlib
from ark.segmentation import ez_object_segmentation, ez_seg_display, merge_masks, composite_builder
from alpineer import load_utils
from tqdm.notebook import tqdm
import xarray as xr

In [ ]:
# set up the base directory
base_dir = "../data/example_dataset"

If you would like to test the features in Ark with an example dataset, run the cell below. It will download a dataset consisting of 11 FOVs with 22 channels. You may find more information about the example dataset in the [README](../README.md#example-dataset).

If you are using your own data, skip the cell below.

* `overwrite_existing`: If set to `False`, it will not overwrite existing data in the `data/example_dataset`. Recommended leaving as `True` if you are doing a clean run of the `ark` pipeline using this dataset from the start. If you already have the dataset downloaded, set to `False`.

In [ ]:
example_dataset.get_example_dataset(dataset="ez_segment_image_data", save_dir = base_dir, overwrite_existing = True)

## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'

If you're interested in directly interfacing with Google Drive, consult the documentation [here](https://ark-analysis.readthedocs.io/en/latest/_rtd/google_docs_usage.html).

In [ ]:
# set up file paths
tiff_dir = os.path.join(base_dir, "image_data")
ez_table_dir = os.path.join(base_dir, "segmentation/ez_table")
ez_input_dir = os.path.join(base_dir, "segmentation/ez_input")
ez_output_dir = os.path.join(base_dir, "segmentation/ez_output")
ez_visualization_dir = os.path.join(base_dir, "segmentation/ez_visualization")

In [ ]:
# create directories if do not exist
for directory in [ez_table_dir, ez_input_dir, ez_output_dir, ez_visualization_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# validate paths
io_utils.validate_paths([base_dir,
                         tiff_dir,
                         ez_input_dir,
                         ez_output_dir,
                         ez_table_dir,
                         ez_visualization_dir
                         ])

### Compute and filter fov paths

In [ ]:
# either get all fovs in the folder...
fovs = io_utils.list_folders(tiff_dir)

# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

## 2. Composite Builder

#### Set composite paths, test FoV

In [45]:
# Give path names for where composite images should be stored
composite_directory = "../../data/experiments/ezSeg Data/Composites/Point77/"
# What would you like to name your composite image
composite_name = "composite_a.tiff"

In [ ]:
# What channels would you like to add together?
to_add = ["8OHGuano", "C12", "Reelin"]
# What channels would you like to subtract?
to_subtract = ["MBP", "Iba1", "Si28"]

#### Create your composite channel.
Specify image_type ("signal" or "pixel_clustered"), and composite_method ("total" or "binary") below.

In [ ]:
# Run composite builder
composite_array = composite_builder.composite_builder(
    data=object_masks,
    images_to_add=to_add,
    images_to_subtract=to_subtract,
    composite_name = composite_name,
    composite_directory=composite_directory,
    image_type="signal",
    composite_method="total",
)


In [ ]:
# Specify which FoV you'd like to see for visual testing purposes.
test_FoV_name = ""
composite_test_image = "" # this should take the base path from earlier, the composite path, the test FoV name, and the composite name without user input.
# Show test composite image
ez_seg_display.display_channel_image(composite_test_image)

## 3. Create Object Masks

#### Create your object segmentation masks.
Here you will input which channel you would like as a base for segmenting single object masks.

Additionally, set segmentation parameters below, including: object type, blur, threshold, smallest hole size allowed, fov size (one side, in um),
and minimum & maximum object areas of the object masks.

In [ ]:
# Name the channel you want to segment on.
channel_to_segment = ""
################################# need to pull the channel from the xarray. Alternatively need to only load test images and run on all afterwards.
# object shape (blur or projection)
object_shape = "blob"
# blurring value
blur = "1"
# signal value to start mask creation on. None defaults to a local thresholding value.
threshold = None
# any holes above the hole_size value will be filled in objects. None defaults to a hole size value based on the fov and number of pixels.
hole_size = None
# the length of one side of your fov in um.
fov_size = 400
# minimum number of pixels required in a segmented object
min_pixels = 100
# maximum number of pixels required in a segmented object
max_pixels = 100000

In [ ]:
# Input parameters and segment images.
create_object_masks(
    input_image: np.ndarray,
    object_shape_type: str = object_shape,
    sigma: int = blur,
    thresh: Optional[np.float32] = None,
    hole_size: Optional[int] = None,
    fov_dim: int = 400,
    min_object_area: int = 100,
    max_object_area: int = 100000,
)


In [ ]:
# Specify which segmented FoV you'd like to see for visual testing purposes.
test_FoV_name = ""
ez_seg_test_masks = "" # this should take the base path from earlier, the composite path, the test FoV name, and the composite name without user input.
# Show test composite image
ez_seg_display.overlay_mask_outlines(channel_to_segment, ez_seg_test_masks)

## 4. Mask Merger

Merging enables connecting traditional circular or oval shaped nucelar-based cell masks with anuclear cell projections (e.g. microglia arms with microglia soma)
**Note:** Requires the Deepcell outputs from `1_Segment_Image_Data.ipynb`.

#### Here you can merge object segmentation masks with cell masks (or any other type of mask).
Here you will provide a list of what objects you would like to merge with previously segmented cell masks (or other base mask).

**LIST ORDER IMPORTANT**: The first mask listed will be merged first, the second mask with cells not merged during the first merge, etc.

Additionally, set the percent area of an object that needs to be overlapping onto a cell mask to get merged.

In [25]:
# Input path
curr_object_mask = pathlib.Path("../../data/experiments/ezSeg Data/segmentation/deepcell_output/TIFs_whole_cell.tiff")
# List of object masks to merge to the base (cell) image.
merge_masks_list = ["microglia-arms", "astrocyte-arms", "neuropil"]
# Value (1-100) required for an object mask to be merged into a cell mask.
percent_overlap = 70

In [ ]:
# Decide here if you want a single mask combining all cells and objects found in merge_mask_list (True) or a merged mask saved for each cell and object type (False).
fully_combined_masks = True

In [62]:
merge_masks.merge_masks_seq(
    object_list=["composite_b", "composite_a"],
    object_mask_dir=ez_directory,
    cell_mask_path=curr_object_mask,
    overlap=percent_overlap,
    save_path=composite_directory,
)

In [ ]:
# # Specify which segmented FoV you'd like to see for visual testing purposes.
test_FoV_name = ""
ez_seg_test_masks = "" # this should take the base path from earlier, the composite path, the test FoV name, and the composite name without user input.
# Show test composite image
ez_seg_display.multiple_mask_displays(merge_masks_list, cell_mask)

## 5. Generate single cell and/or object expression table

In [ ]:
# set to True to bypass expensive cell or object property calculations
# only cell or object label, size, and centroid will be extracted if True
fast_extraction = False

# If you want to give your cell label an alternative name (e.g. plaques.csv), write that name below, otherwise leave as "cell".
table_name = "cell"

For a full list of features extracted, please refer to the cell table section of: https://ark-analysis.readthedocs.io/en/latest/_rtd/data_types.html

In [ ]:
# now extract the segmented imaging data to create normalized and transformed expression matrices
# note that if you're loading your own dataset, please make sure all the imaging data is in the same folder
# with each fov given its own folder and all fovs having the same channels
cell_table_size_normalized, cell_table_arcsinh_transformed = \
    marker_quantification.generate_cell_table(segmentation_dir=deepcell_output_dir,
                                              tiff_dir=tiff_dir,
                                              img_sub_folder=None,
                                              fovs=fovs,
                                              batch_size=5,
                                              nuclear_counts=nuclear_counts,
                                              fast_extraction=fast_extraction)

In [ ]:
# Set the compression level if desired, ZSTD compression can offer up to a 60-70% reduction in file size.
# NOTE: Compressed `csv` files cannot be opened in Excel. They must be uncompressed beforehand.
compression = None

# Uncomment the line below to allow for compressed `csv` files.
# compression = {"method": "zstd", "level": 3}

cell_table_size_normalized.to_csv(os.path.join(cell_table_dir, table_name + '_table_size_normalized.csv'),
                                  compression=compression, index=False)
cell_table_arcsinh_transformed.to_csv(os.path.join(cell_table_dir, table_name + '_table_arcsinh_transformed.csv'),
                                      compression=compression, index=False)

## End - Cell &| Object Masks and Table have been saved